In [3]:
import os
os.environ['OMP_NUM_THREADS'] = '1'
import mindspore as ms
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target='CPU')

from mindquantum import *
import numpy as np
from mindquantum.nn import MQAnsatzOnlyLayer
from mindquantum.ansatz import MaxCutAnsatz
from mindspore.nn import Adam
from mindspore.nn import TrainOneStepCell

In [7]:
edge_dict ={0: [1, 2, 3, 11],
            1: [0, 3, 6, 8, 9, 10, 11],
            2: [0, 6, 8, 10, 13],
            3: [0, 1, 12, 14],
            4: [7, 9],
            5: [6, 7, 12],
            6: [1, 2, 5, 7],
            7: [4, 5, 6, 10, 11, 12],
            8: [1, 2, 10, 13, 14],
            9: [1, 4, 14],
            10: [1, 2, 7, 8, 14],
            11: [0, 1, 7],
            12: [3, 5, 7, 14],
            13: [2, 8],
            14: [3, 8, 9, 10, 12]}
graph = []
for key in edge_dict.keys():
    for val in edge_dict[key]:
        if key <= val:
            graph.append((key, val))

In [8]:
maxcut = MaxCutAnsatz(graph, 12)
sim = Simulator('projectq', maxcut.circuit.n_qubits)
grad_ops = sim.get_expectation_with_grad(Hamiltonian(-maxcut.hamiltonian), maxcut.circuit)
net = MQAnsatzOnlyLayer(grad_ops)

opti = Adam(net.trainable_params(), learning_rate=0.1)
train_net = TrainOneStepCell(net, opti)
for i in range(150):
    res = train_net()
    if i % 10 == 0:
        print(f"setp: {i}, res: {-res}")

setp: 0, res: [15.559407]
setp: 10, res: [21.690279]
setp: 20, res: [22.476614]
setp: 30, res: [23.199774]
setp: 40, res: [23.334793]
setp: 50, res: [23.427404]
setp: 60, res: [23.454636]
setp: 70, res: [23.483658]
setp: 80, res: [23.520897]
setp: 90, res: [23.54464]
setp: 100, res: [23.553316]
setp: 110, res: [23.569418]
setp: 120, res: [23.6387]
setp: 130, res: [23.67979]
setp: 140, res: [23.690142]
